In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

from sklearn.ensemble import BaggingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
X_pumps = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv')
print(f'X_pumps: {X_pumps.shape}')

y_pumps = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')
print(f'y_pumps: {y_pumps.shape}')

X_test = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')
print(f'X_test: {X_test.shape}')

X_pumps: (59400, 40)
y_pumps: (59400, 2)
X_test: (14850, 40)


In [4]:
X_pumps.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [5]:
y_pumps = y_pumps.set_index('id')
y_pumps.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


In [6]:
X_test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


## Combine the training and test datasets so we can impute features with missing values, normalize features, etc.
* Use ignore_index and reset_index so that we can sort the rows back into their original order when we split the training and test datasets apart.

In [7]:
X_combined = pd.concat([X_pumps, X_test], ignore_index=True, axis=0)
X_combined = X_combined.reset_index()
print(f'X_combined: {X_combined.shape}')

X_combined: (74250, 41)


## Several features have values which imply a missing value
* Replace these values with NaN to make them easier to impute

In [8]:
def replace_none_with_nan(df):
    df.loc[df.longitude == 0, 'latitude'] = np.nan
    df.loc[df.longitude == 0, 'longitude'] = np.nan
    df.loc[df.gps_height == 0, 'gps_height'] = np.nan    
    df.loc[df.population == 0, 'population'] = np.nan    
    df.loc[df.amount_tsh == 0, 'amount_tsh'] = np.nan
    df.loc[df.construction_year == 0, 'construction_year'] = np.nan
    df.loc[df.funder == '0', 'funder'] = np.nan
    df.loc[df.installer.isin(['0', '-']), 'installer'] = np.nan
    return df

In [9]:
X_combined = replace_none_with_nan(X_combined)

## Some categorical features have string values with lower case, mixed case, and upper case
* Convert these to upper case

In [10]:
def cleanup_categorical_features(df):
    df.installer = df.installer.str.upper()
    df.funder = df.funder.str.upper()
    df.scheme_name = df.scheme_name.str.upper()
    return df

In [11]:
X_combined = cleanup_categorical_features(X_combined)

## Some boolean features have missing values
* Impute them using the most common value

In [12]:
def cleanup_boolean_features(df):
    df.public_meeting = df.public_meeting.fillna(df.public_meeting.median()).astype(int)
    df.permit = df.permit.fillna(df.permit.median()).astype(int)
    return df

In [13]:
X_combined = cleanup_boolean_features(X_combined)

## Impute latitude/longitude missing values
* Use centroid of region/district/lga/ward

In [14]:
def get_imputer_lat_long(df, group_columns):
    grp = df.groupby(group_columns)
    imputer = grp[['latitude', 'longitude']].mean()
    return imputer

In [15]:
def impute_lat_long(df, impute_df, group_columns):
    df = pd.merge(df, 
                  impute_df, 
                  left_on=group_columns,
                  right_index=True, 
                  suffixes=('', '_mean'))
    
    df.loc[df.latitude.isnull(), 'latitude'] = df.loc[df.latitude.isnull(), 'latitude_mean']
    df.loc[df.longitude.isnull(), 'longitude'] = df.loc[df.longitude.isnull(), 'longitude_mean']
    df.drop(['latitude_mean', 'longitude_mean'], axis=1, inplace=True)
    return df

In [16]:
def impute_by_centroid(df, group_columns):
    while len(group_columns) > 0:
        imp = get_imputer_lat_long(df, group_columns=group_columns)
        df = impute_lat_long(df, imp, group_columns=group_columns)
        
        group_columns.pop()
        
    return df

## Impute numeric features with missing values
* Use median of basin/region/district/lga/ward

In [17]:
def get_feature_imputer_median(df, group_columns, feature_column):
    grp = df.groupby(group_columns)
    imputer = grp[feature_column].median().round()
    return imputer

In [18]:
def impute_feature_by_median(df, impute_df, group_columns, feature_column):
    df = pd.merge(df, 
                  impute_df.to_frame(), 
                  left_on=group_columns, 
                  right_index=True, 
                  suffixes=('', '_median'))
    
    df.loc[df[feature_column].isnull(), feature_column] = df.loc[df[feature_column].isnull(), 
                                                                        feature_column + '_median']
    
    df.drop(feature_column + '_median', axis=1, inplace=True)
    return df

In [19]:
def impute_by_median(df, group_columns, feature_column):    
    while len(group_columns) > 0:
        imp = get_feature_imputer_median(df, 
                                         group_columns=group_columns, 
                                         feature_column=feature_column)

        df = impute_feature_by_median(df, 
                                      imp,
                                      group_columns=group_columns,
                                      feature_column=feature_column)

        group_columns.pop()
    
    return df

## Impute categorical features with missing values
* Use most common value of basin/region/district/lga/ward

In [20]:
def get_feature_imputer_size(df, group_columns, feature_column):
    grp = df.groupby(group_columns + [feature_column]).size()
    grp.rename('size', inplace=True)
    idxmax = grp.reset_index().groupby(group_columns)['size'].idxmax()
    feature = grp[idxmax].to_frame()
    feature = feature.drop('size', axis=1)
    imputer = feature.reset_index(feature_column)
    return imputer

In [21]:
def impute_feature_by_size(df, impute_df, group_columns, feature_column):
    df = pd.merge(df, 
                  impute_df, 
                  how='left',
                  left_on=group_columns, 
                  right_index=True, 
                  suffixes=('', '_top'))
    
    df.loc[df[feature_column].isnull(), feature_column] = df.loc[df[feature_column].isnull(), feature_column + '_top']

    df.drop(feature_column + '_top', axis=1, inplace=True)
    return df

In [22]:
def impute_by_size(df, group_columns, feature_column):   
    while len(group_columns) > 0:
        imp = get_feature_imputer_size(df, 
                                       group_columns=group_columns, 
                                       feature_column=feature_column)

        df = impute_feature_by_size(df,
                                    imp,
                                    group_columns=group_columns,
                                    feature_column=feature_column)

        group_columns.pop()
    
    return df

### Latitude and Longitude

In [23]:
group_columns = ['region_code', 'district_code', 'lga', 'ward']
X_combined = impute_by_centroid(X_combined, group_columns)

### Construction Year

In [24]:
group_columns = ['basin', 'region_code', 'district_code', 'lga', 'ward']
feature_column = 'construction_year'
X_combined = impute_by_median(X_combined, group_columns, feature_column)

### Population

In [25]:
group_columns = ['basin', 'region_code', 'district_code', 'lga', 'ward']
feature_column = 'population'
X_combined = impute_by_median(X_combined, group_columns, feature_column)

### GPS Height

In [26]:
group_columns = ['basin', 'region_code', 'district_code', 'lga', 'ward']
feature_column = 'gps_height'
X_combined = impute_by_median(X_combined, group_columns, feature_column)

### Amount TSH

In [27]:
group_columns = ['basin', 'region_code', 'district_code', 'lga', 'ward']
feature_column = 'amount_tsh'
X_combined = impute_by_median(X_combined, group_columns, feature_column)

### Scheme Management

In [28]:
group_columns = ['basin', 'region_code', 'district_code', 'lga', 'ward']
feature_column = 'scheme_management'
X_combined = impute_by_size(X_combined, group_columns, feature_column)

## Features to use

In [29]:
all_features = ['amount_tsh', 'basin', 'construction_year', 'date_recorded',
       'district_code', 'extraction_type', 'extraction_type_class',
       'extraction_type_group', 'funder', 'gps_height', 'installer',
       'latitude', 'lga', 'longitude', 'management', 'management_group',
       'num_private', 'payment', 'payment_type', 'permit', 'population',
       'public_meeting', 'quality_group', 'quantity', 'quantity_group',
       'recorded_by', 'region', 'region_code', 'scheme_management',
       'scheme_name', 'source', 'source_class', 'source_type', 'subvillage',
       'ward', 'water_quality', 'waterpoint_type', 'waterpoint_type_group',
       'wpt_name']

numeric_features = ['amount_tsh', 'construction_year', 'gps_height',
                    'latitude', 'longitude', 'population']

boolean_features = ['permit', 'public_meeting']

categorical_features = ['extraction_type_class', 'extraction_type_group', 
                        'management', 'management_group',  'payment', 'quality_group',
                        'quantity', 'scheme_management', 'source_class', 'source', 
                        'waterpoint_type']

drop_features = list(set(all_features) - set(numeric_features) \
                     - set(boolean_features) - set(categorical_features))

print('drop_features:\n', sorted(drop_features))

drop_features:
 ['basin', 'date_recorded', 'district_code', 'extraction_type', 'funder', 'installer', 'lga', 'num_private', 'payment_type', 'quantity_group', 'recorded_by', 'region', 'region_code', 'scheme_name', 'source_type', 'subvillage', 'ward', 'water_quality', 'waterpoint_type_group', 'wpt_name']


## Confirm no missing values on features to be used

In [30]:
def confirm_no_missing_values(df):
    print(df[numeric_features + boolean_features + \
             categorical_features].isnull().sum().sort_index())

In [31]:
confirm_no_missing_values(X_combined)

amount_tsh               0
construction_year        0
extraction_type_class    0
extraction_type_group    0
gps_height               0
latitude                 0
longitude                0
management               0
management_group         0
payment                  0
permit                   0
population               0
public_meeting           0
quality_group            0
quantity                 0
scheme_management        0
source                   0
source_class             0
waterpoint_type          0
dtype: int64


## Drop features not going to be used

In [32]:
def drop_features_not_used(df):
    df.drop(drop_features, axis=1, inplace=True)

In [33]:
print(f'X_combined: {X_combined.shape}')
drop_features_not_used(X_combined)
print(f'X_combined: {X_combined.shape}')

X_combined: (74250, 41)
X_combined: (74250, 21)


## Turn categorical features into dummy variables

In [34]:
def categorical_features_to_dummies(df):
    df = pd.get_dummies(df, 
                        drop_first=True, 
                        prefix=categorical_features, 
                        columns=categorical_features)
    
    return df

In [35]:
print(f'X_combined: {X_combined.shape}')
X_combined = categorical_features_to_dummies(X_combined)
print(f'X_combined: {X_combined.shape}')

X_combined: (74250, 21)
X_combined: (74250, 86)


## Normalize the feature values

In [36]:
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

In [37]:
X_combined[numeric_features] = X_combined[numeric_features].apply(normalize, axis=0)

## Restore the dataset rows back to their original order and make the id column the index 
* Various dataframe operations cause the dataset order to change

In [38]:
X_combined = X_combined.sort_values(by='index')
X_combined = X_combined.set_index('id')
X_combined = X_combined.drop('index', axis=1)
X_combined.head()

,amount_tsh,gps_height,longitude,latitude,population,public_meeting,permit,construction_year,extraction_type_class_handpump,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,extraction_type_group_gravity,extraction_type_group_india mark ii,extraction_type_group_india mark iii,extraction_type_group_mono,extraction_type_group_nira/tanira,extraction_type_group_other,extraction_type_group_other handpump,extraction_type_group_other motorpump,extraction_type_group_rope pump,extraction_type_group_submersible,extraction_type_group_swn 80,extraction_type_group_wind-powered,management_other,management_other - school,management_parastatal,management_private operator,management_trust,management_unknown,management_vwc,management_water authority,management_water board,management_wua,management_wug,management_group_other,management_group_parastatal,management_group_unknown,management_group_user-group,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucket,payment_pay when scheme fails,payment_unknown,quality_group_fluoride,quality_group_good,quality_group_milky,quality_group_salty,quality_group_unknown,quantity_enough,quantity_insufficient,quantity_seasonal,quantity_unknown,scheme_management_None,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,source_class_surface,source_class_unknown,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,1.448346,0.515729,-0.064889,-1.444812,-0.392397,1,0,0.171913,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
8776,-0.088812,0.532126,-0.157602,1.292976,-0.030284,1,1,1.233508,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
34310,-0.221567,-0.766894,0.912340,0.698506,-0.093813,1,1,1.137000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
67743,-0.214580,-1.537561,1.309610,-1.906141,-0.500396,1,1,-1.082700,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
19728,-0.144709,0.273415,-1.539792,1.407372,0.435592,1,1,-0.117613,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


## Split the combined dataset back into training and test datasets

In [39]:
X_pumps = X_combined.iloc[:len(X_pumps)]
X_test = X_combined.iloc[len(X_pumps):]
print(f'X_pumps: {X_pumps.shape}')
print(f'X_test: {X_test.shape}')

del X_combined

X_pumps: (59400, 84)
X_test: (14850, 84)


## Split training dataset into train and cross validation sets

In [40]:
X_train, X_cross, y_train, y_cross = train_test_split(X_pumps, 
                                                      y_pumps,
                                                      test_size=0.2,
                                                      stratify=y_pumps.status_group)

## Try dummy classifier for baseline

In [41]:
def train_predict_score(clf, X, y):
    clf.fit(X, y.status_group)
    return clf.score(X, y)

In [42]:
clf = DummyClassifier(strategy='most_frequent', random_state=67)
train_predict_score(clf, X_train, y_train)

0.54307659932659935

## Try various classification algorithms to see which look most promising
* Logistic Regression
* KNN
* Random Forest
* Gradient Boosting
* Bagging
* MLP (neural network)

In [43]:
def train_cross_val_score(clf, X, y, scoring):
    clf.fit(X_train, y_train.status_group)
    scores = cross_val_score(clf, X, y.status_group, scoring=scoring)
    print("Scores: {}".format(scores))
    print("Avg Score: {}".format(scores.mean()))

### Logistic Regression

In [44]:
%%time
clf = LogisticRegression(random_state=67, n_jobs=-1)
train_cross_val_score(clf, X_train, y_train, 'accuracy')

Scores: [ 0.72844338  0.72908643  0.7343898 ]
Avg Score: 0.7306398693070885
CPU times: user 6.8 s, sys: 219 ms, total: 7.02 s
Wall time: 6.69 s


### KNN

In [45]:
%%time
clf = KNeighborsClassifier(n_jobs=2)
train_cross_val_score(clf, X_train, y_train, 'accuracy')

Scores: [ 0.75558642  0.75989646  0.76002273]
Avg Score: 0.7585018675579779
CPU times: user 36.1 s, sys: 230 ms, total: 36.3 s
Wall time: 19.4 s


### Random Forest

In [46]:
%%time
clf = RandomForestClassifier(random_state=67)
train_cross_val_score(clf, X_train, y_train, 'accuracy')

Scores: [ 0.77938392  0.77908959  0.78243576]
Avg Score: 0.7803030883315758
CPU times: user 2.4 s, sys: 123 ms, total: 2.52 s
Wall time: 2.52 s


### Gradient Boosting

In [47]:
%%time
clf = GradientBoostingClassifier(n_estimators=30, random_state=67)
train_cross_val_score(clf, X_train, y_train, 'accuracy')

Scores: [ 0.72749653  0.73142244  0.73306396]
Avg Score: 0.7306609741864039
CPU times: user 30.4 s, sys: 426 ms, total: 30.8 s
Wall time: 31.2 s


### Bagging

In [48]:
%%time
clf = BaggingClassifier(random_state=67, n_jobs=2)
train_cross_val_score(clf, X_train, y_train, 'accuracy')

Scores: [ 0.7761015   0.77978408  0.77612223]
Avg Score: 0.7773359365173095
CPU times: user 1.07 s, sys: 494 ms, total: 1.57 s
Wall time: 9.37 s


### MLP

In [49]:
%%time
clf = MLPClassifier(random_state=67)
train_cross_val_score(clf, X_train, y_train, 'accuracy')

Scores: [ 0.76726423  0.76823032  0.76860913]
Avg Score: 0.7680345604162101
CPU times: user 2min 1s, sys: 15.2 s, total: 2min 16s
Wall time: 1min 10s


## Use grid search to determine optimal hyper-parameters for best performing models

In [50]:
def grid_search_cross_val(clf, X, y, param_grid, scoring):
    grid = GridSearchCV(clf, param_grid=param_grid, scoring=scoring)
    grid.fit(X, y.status_group)
    print("Best score: {}".format(grid.best_score_))
    print("Best params: {}".format(grid.best_params_))

### Random Forest

In [51]:
%%time
clf = RandomForestClassifier(random_state=67, max_depth=None)

param_grid = [
    {
        'n_estimators': [30, 100, 300],
    } 
]

grid_search_cross_val(clf, X_train, y_train, param_grid, scoring='accuracy')

Best score: 0.7919402356902356
Best params: {'n_estimators': 300}
CPU times: user 1min 46s, sys: 4.39 s, total: 1min 50s
Wall time: 1min 52s


### Bagging

In [52]:
%%time
clf = BaggingClassifier(random_state=67, n_jobs=1)

param_grid = [
    {'n_estimators': [30, 100, 300]}
]

grid_search_cross_val(clf, X_train, y_train, param_grid, scoring='accuracy')

Best score: 0.7871001683501684
Best params: {'n_estimators': 300}
CPU times: user 8min 10s, sys: 25.3 s, total: 8min 36s
Wall time: 8min 43s


# Cross validate using the out of sample datasets

In [53]:
def cross_val_out_of_sample(clf, X_train, y_train, X_cross, y_cross):
    clf.fit(X_train, y_train.status_group)
    y_pred = clf.predict(X_cross)
    print(accuracy_score(y_true=y_cross, y_pred=y_pred))
    return y_pred

### Random Forest

In [54]:
%%time
clf = RandomForestClassifier(max_depth=None, 
                             n_estimators=300,
                             random_state=67)

y_pred = cross_val_out_of_sample(clf, X_train, y_train, X_cross, y_cross)

0.804545454545
CPU times: user 24.3 s, sys: 711 ms, total: 25 s
Wall time: 25.3 s


### Bagging

In [55]:
%%time
clf = BaggingClassifier(n_estimators=300, random_state=67, n_jobs=1)

y_pred = cross_val_out_of_sample(clf, X_train, y_train, X_cross, y_cross)

0.802104377104
CPU times: user 2min 15s, sys: 5.67 s, total: 2min 20s
Wall time: 2min 23s


# Prepare test dataset
* Retrain classifier on entire train dataset

In [56]:
X_test.head()

,amount_tsh,gps_height,longitude,latitude,population,public_meeting,permit,construction_year,extraction_type_class_handpump,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,extraction_type_group_gravity,extraction_type_group_india mark ii,extraction_type_group_india mark iii,extraction_type_group_mono,extraction_type_group_nira/tanira,extraction_type_group_other,extraction_type_group_other handpump,extraction_type_group_other motorpump,extraction_type_group_rope pump,extraction_type_group_submersible,extraction_type_group_swn 80,extraction_type_group_wind-powered,management_other,management_other - school,management_parastatal,management_private operator,management_trust,management_unknown,management_vwc,management_water authority,management_water board,management_wua,management_wug,management_group_other,management_group_parastatal,management_group_unknown,management_group_user-group,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucket,payment_pay when scheme fails,payment_unknown,quality_group_fluoride,quality_group_good,quality_group_milky,quality_group_salty,quality_group_unknown,quantity_enough,quantity_insufficient,quantity_seasonal,quantity_unknown,scheme_management_None,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,source_class_surface,source_class_unknown,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50785,1.168863,1.619805,0.071748,0.613850,0.056538,1,1,1.426526,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
51630,-0.214580,0.841850,0.600892,0.880383,0.012068,1,1,0.268422,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
17168,-0.172657,0.838207,-0.130835,0.278360,0.435592,1,1,1.233508,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
45559,-0.214580,-1.530273,1.143762,-1.289381,-0.093813,1,1,-0.986191,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
49871,-0.088812,0.278881,-0.038534,-1.833376,-0.496160,1,1,0.268422,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


## Train the classifier with the full training dataset

In [57]:
clf = RandomForestClassifier(max_depth=None, n_estimators=300)
clf.fit(X_pumps, y_pumps.status_group)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

## Predict whether the water pumps will fail

In [58]:
y_pred = clf.predict(X_test)
predict_df = pd.DataFrame({'status_group': y_pred}, index=X_test.index)
predict_df.head()

,status_group
id,
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [59]:
submission_filename = 'water_pump_submission_000.csv' 
predict_df.to_csv(submission_filename, index=True)

## Check if submission file matches template file

In [60]:
df1 = pd.read_csv('SubmissionFormat.csv', usecols=[0], header=0, names=['format_id'])
df2 = pd.read_csv(submission_filename, usecols=[0], header=0, names=['submit_id'])
df = pd.merge(df1, df2, how='left', left_index=True, right_index=True)
df[df.format_id != df.submit_id]

,format_id,submit_id
